# Feature Engineering the Github Data

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# read the data
df = pd.read_csv("../data/total.csv")

In [3]:
# checking the data shape
df.shape

(601077, 35)

In [5]:
df

,sha,message,comment_count,date,total_deletions,total_additions,total,comments,additions,deletions,...,forks,open_issues,watchers,network_count,type,admin,push,pull,label,owner
0,cba2407f4a86c0a040fe0473615f9987ab108b9c,Use github's new relative path format.,0,2013-01-31T04:46:46Z,3,2,5,[],[2],[3],...,41,0,298,41,User,False,False,True,bug,mavam
1,758a5bae6d25eb3f67a28537dd5c9c3db2648368,Fix conditional variance of LS estimate.\n\nIn...,0,2013-01-31T04:32:15Z,2,3,5,[],[3],[2],...,41,0,298,41,User,False,False,True,bug,mavam
2,7e5a9a426668b14b4fd22b227b54446185667649,Merge pull request #8 from cortex/master\n\nSh...,0,2013-02-03T03:03:09Z,0,15,15,[],"[9, 6]","[0, 0]",...,27,2,194,27,User,False,False,True,NaN,NaN
3,3ea807b85160061d0f43dcf2f8bed5311da04edf,Integrate Mathieu Bryen's pull request.,0,2012-10-10T15:35:05Z,0,8,8,[],[8],[0],...,41,0,298,41,User,False,False,True,bug,mavam
4,15b3e5d8c552adbd5ee5da0aa0b029197fa3a69d,Merge branch 'master' of github.com:mavam/stat...,0,2012-10-10T15:32:28Z,1,1,2,[],[1],[1],...,41,0,298,41,User,False,False,True,bug,mavam
5,bcf66b6bd7e32f4f853e1762893307c4674eab29,Merge pull request #10 from cortex/mysql-utf8\...,0,2013-02-03T03:01:33Z,0,1,1,[],[1],[0],...,27,2,194,27,User,False,False,True,NaN,NaN
6,2b8be0fbde43d58b6282271f2ceb5f15fcce40db,Merge branch 'mysql-utf8',0,2013-01-29T17:02:48Z,0,1,1,[],[1],[0],...,27,2,194,27,User,False,False,True,NaN,NaN
7,1212b3e3d013f8a99385383bf0502e8e6e6b5185,Use UTF-8 for mysql connections,0,2013-01-29T17:02:15Z,0,1,1,[],[1],[0],...,27,2,194,27,User,False,False,True,NaN,NaN
8,8047099afdfc5c973faa67a04f601a014de0fa73,Please find an active fork for this project if...,0,2012-04-08T02:29:18Z,0,2,2,[],[2],[0],...,803,3,3129,803,User,False,False,True,NaN,NaN
9,54619cf45e30da038a767949b7b0549ed2d791bb,Merge pull request #2 from mathbruyen/patch-1\...,0,2012-10-10T15:31:52Z,1,1,2,[],[1],[1],...,41,0,298,41,User,False,False,True,bug,mavam


### One hot-encoding

In [4]:
# import preprocessing from sklearn
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
# encoding these columns that are 0/1
df['private'] = le.fit_transform(df.private.values)
df['fork'] = le.fit_transform(df.fork.values)
df['has_issues'] = le.fit_transform(df.has_issues.values)
df['has_downloads'] = le.fit_transform(df.has_downloads.values)
df['has_wiki'] = le.fit_transform(df.has_wiki.values)
df['admin'] = le.fit_transform(df.admin.values)
df['push'] = le.fit_transform(df.push.values)
df['pull'] = le.fit_transform(df.pull.values)
df['type'] = le.fit_transform(df.type.values)

In [6]:
# encoding the language categorical feature
df = pd.get_dummies(df, columns = ["language"])

In [7]:
df.shape

(601077, 50)

### Feature Creation

In [8]:
# create a new column called total_files that sums up the filenames column
df['total_files'] = df.apply(lambda row: row.filenames.count(',') + 1, axis=1)

In [9]:
# use that paper to classify the commits as being defective of not
df['results'] = np.where(df['message'].str.lower().str.contains("bug|fix|error|wrong|fail|problem|patch"), 
                         1, 
                         0)

In [10]:
df.shape

(601077, 52)

In [49]:
# save the csv
df.to_csv('../data/total_processed.csv')